## Agentic Memory and Streaming
### Memory & Streaming - Stateful, Responsive Agents

Learning Objectives:
- Implement conversation memory with checkpointers
- Use thread_id for multiple conversations
- Stream responses for real-time UX

#### Real-World Use Cases:
1. Customer Support: Remember customer issues across sessions
2. Personal Assistants: Maintain user preferences and history
3. Educational Tutors: Track learning progress
4. Creative Tools: Continue stories, designs across sessions
5. Code Assistants: Remember project context


In [ ]:

from typing_extensions import TypedDict, Annotated
import operator
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import ToolNode

# Configuration
BASE_URL = "http://localhost:11434"
MODEL_NAME = "qwen3"

llm = ChatOllama(model=MODEL_NAME, base_url=BASE_URL)

In [ ]:
## Tool usage
import sys
sys.path.append(r"../05. LangGraph ReAct Agent with Tools")

import my_tools

all_tools = [my_tools.get_weather, my_tools.calculate]

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list, operator.add]


In [ ]:
# =============================================================================
# Nodes
# =============================================================================

def agent_node(state: AgentState) -> dict:
    """Agent with tools and memory."""
    
    llm_with_tools = llm.bind_tools(all_tools)

    system_message = SystemMessage(content="""You are a friendly assistant with memory and access to documentation search.
        "Use the available tools to help the user when necessary.""")

    messages = [system_message] + state['messages']
        
    # LLM decides whether to use tools or respond directly
    response = llm_with_tools.invoke(messages)
    
    return {"messages": [response]}


In [ ]:
def should_continue(state: AgentState):
    """Route to tools or end."""
    last = state["messages"][-1]
    # If there are tool calls, route to tools
    if hasattr(last, "tool_calls") and last.tool_calls:
        return "tools"
    # Otherwise, we're done
    return END


In [ ]:

# =============================================================================
# Graph
# =============================================================================

def create_agent():
    """Create chatbot with tools, memory, and streaming support."""
    builder = StateGraph(AgentState)

    # Add nodes
    builder.add_node("agent", agent_node)
    builder.add_node("tools", ToolNode(all_tools))

    # Define flow
    builder.add_edge(START, "agent")
    # Conditional routing: tools or end
    builder.add_conditional_edges("agent", should_continue, ["tools", END])
    # After tools, go back to agent
    builder.add_edge("tools", "agent")

    # Add checkpointer for memory across invocations
    checkpointer = MemorySaver()
    # Compile with checkpointer enables memory
    return builder.compile(checkpointer=checkpointer)



In [ ]:
graph = create_agent()
graph

In [ ]:
# Invoke graph with same thread_id to maintain memory
msg = "Hello, My name is Laxmi Kant Tiwari."

# msg = "Can you search the documentation for 'langgraph' and summarize it for me?"
# msg = "Can you search the documentation for 'langchain' and summarize it for me?"

thread_id = "laxmikant"


def chat(msg, thread_id):
    config = {"configurable": {"thread_id": thread_id}}
    for chunk in graph.stream({"messages": [msg]}, config):
        # print("\n--- New Chunk ---\n", chunk)

        if 'agent' in chunk:
            chunk = chunk.get('agent')
        else:
            chunk = chunk.get('tools')
            
        # check chunk is tool message
        if hasattr(chunk["messages"][-1], "tool_calls") and chunk["messages"][-1].tool_calls:
            for tc in chunk["messages"][-1].tool_calls:
                # tool name and arguments
                print(f"\n[TOOL CALL] {tc.get('name')} with args {tc.get('args')}")
        else:
            print(f"\n[AGENT] {chunk['messages'][-1].content}")

chat(msg, thread_id)

In [ ]:
msg = "what is my name? and summarize the previous answer."
thread_id = "laxmikant"

chat(msg, thread_id)

In [ ]:
msg = "what is my name? and summarize the previous answer."
thread_id = "kgptalkie"

chat(msg, thread_id)


In [ ]:
msg = "what is my name? and summarize the previous answer."
thread_id = "laxmikant"

chat(msg, thread_id)